In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
df = pd.read_csv("./data/grass_rep.csv", index_col=0)
df

,straw,rice,pasture,feed,weed
Control,5,4,0,-8,4
NaN,17,6,-2,0,0
A,20,10,11,4,16
NaN,38,24,17,0,24
B,10,13,22,15,19
NaN,6,15,18,25,33
C,19,9,8,9,8
NaN,27,7,8,13,2
D,29,15,10,-1,15
NaN,23,25,12,11,19


In [3]:
# np.arrayへ変換 今思えば並べ替えればよかった。わかりづらい。
arr = df.values
arr = arr.reshape(5,2,5)
arr

array([[[ 5,  4,  0, -8,  4],
        [17,  6, -2,  0,  0]],

       [[20, 10, 11,  4, 16],
        [38, 24, 17,  0, 24]],

       [[10, 13, 22, 15, 19],
        [ 6, 15, 18, 25, 33]],

       [[19,  9,  8,  9,  8],
        [27,  7,  8, 13,  2]],

       [[29, 15, 10, -1, 15],
        [23, 25, 12, 11, 19]]], dtype=int64)

In [4]:
# 水準の数と平均
a = arr.shape[0]
r = arr.shape[1]
b = arr.shape[2]
a_mean = arr.mean(axis=2).mean(axis=1) # 行の平均(Control,A-D)
b_mean = arr.mean(axis=0).mean(axis=0) # 列の平均(straw,riceなど)
ab_mean = arr.mean(axis=1) # 個々の水準ごとの平均(r個の平均)
t_mean = arr.mean() # 全部の平均
t_mean

12.68

In [5]:
# 平方和
Sa = b * r * ((a_mean - t_mean)**2).sum()
Sb = a * r * ((b_mean - t_mean)**2).sum()
Sab = 0
for i in range(a):
    for j in range(b):
        Sab += r * (ab_mean[i][j] - a_mean[i] - b_mean[j] + t_mean)**2
Se = ((arr - ab_mean.reshape(5,1,5))**2).sum()
print("Sa:", round(Sa, 2), "Sb:", round(Sb, 2), "Sab:", round(Sab, 2), "Se:", round(Se, 2))

Sa: 1522.08 Sb: 866.88 Sab: 1465.92 Se: 810.0


In [6]:
# 自由度
phi_a = a-1
phi_b = b-1
phi_ab = a*b - a - b + 1 
phi_e = a*b*r - a*b
print("phi_a:",round(phi_a, 2))
print("phi_b:",round(phi_b, 2))
print("phi_ab:",round(phi_ab, 2))
print("phi_e:",round(phi_e, 2))

phi_a: 4
phi_b: 4
phi_ab: 16
phi_e: 25


In [7]:
#平均変動
Va = Sa/phi_a
Vb = Sb/phi_b
Vab = Sab/phi_ab
Ve = Se/phi_e
print("Va:", round(Va, 2), "Vb:", round(Vb, 2), "Vab:", round(Vab, 2), "Ve:", round(Ve, 2))

Va: 380.52 Vb: 216.72 Vab: 91.62 Ve: 32.4


In [8]:
#F値
Fa = Va/Ve
Fb = Vb/Ve
Fab = Vab/Ve
print("Fa:", round(Fa, 2), "Fb:", round(Fb, 2), "Fab:", round(Fab, 2))

Fa: 11.74 Fb: 6.69 Fab: 2.83


In [9]:
alpha = 0.05
print("A:自由度(", phi_a, ",", phi_e, ")のF分布上側", alpha*100, "%点", round(st.f.ppf(1-alpha, phi_a, phi_e), 2))
print("B:自由度(", phi_b, ",", phi_e, ")のF分布上側", alpha*100, "%点", round(st.f.ppf(1-alpha, phi_b, phi_e), 2))
print("AB:自由度(", phi_ab, ",", phi_e, ")のF分布上側", alpha*100, "%点", round(st.f.ppf(1-alpha, phi_ab, phi_e), 2))

A:自由度( 4 , 25 )のF分布上側 5.0 %点 2.76
B:自由度( 4 , 25 )のF分布上側 5.0 %点 2.76
AB:自由度( 16 , 25 )のF分布上側 5.0 %点 2.07
